<a href="https://colab.research.google.com/github/saicharan7479/pytorch-notes/blob/main/petrol_price_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this text is ai generated i mean i wrote it and it removed errors and made it readable :)

First, we need to take the bias and weight parameters.

Then, we sort the data from oldest to newest.

Next, we remove any null values using the dropna() function.

After that, we need to arrange the data line by line to make it easier to normalize.

We then normalize the data (normalization logic taken from ChatGPT).

Once normalized, we convert the data into tensors.

We split the data using the 80/20 rule — 80% for training and 20% for testing.

Next, we define the data (training and test datasets).

After defining the data, we add a loss function and an optimizer:

Why do we use a loss function?
To quantify the error between the model's predictions and the actual target values.

What does the optimizer do?
It tries to minimize the loss function by adjusting the model's parameters.

Note: Loss function, cost function — these terms are often used interchangeably.

Then, we train the model.

I trained the model for 300 predictions, with evaluations at 50, 100, 150, 200, and 250.

After training, we move to the evaluation phase:

model.eval() is used to switch the model to evaluation mode, stopping any further training.

We use with torch.inference_mode(): to make predictions without tracking gradients, which makes it faster and more memory-efficient.



In [ ]:
#petrol predictor by using machine learning
import torch
import torch.nn as nn
import pandas as pd

In [ ]:
df = pd.read_csv("/content/petrol[1].csv")

In [ ]:
#in normal their will be start stop and step we did -1 because we need data from oldest to newest the current data is from newest to oldest
df = df.iloc[::-1].reset_index(drop=True)

In [ ]:
#this shift operation goes down in the row line by line
df["Delhi_yesterday"] = df["Delhi"].shift(1)

In [ ]:
# removing null values
df = df.dropna()

In [ ]:
#this are like bias and weight were we have seen at daniel video
X = df[["Delhi_yesterday", "Kolkata", "Mumbai", "Chennai"]].values
y = df[["Delhi"]].values

In [ ]:
# Manual normalization (Min-Max)
X_min = X.min(axis=0)
X_max = X.max(axis=0)     # got error here so used used normalization here the colab itself recommended this thing, it basically sort things from top to bottom
X_scaled = (X - X_min) / (X_max - X_min)

In [ ]:
# converting into tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32) #we take this here as float32 beacuse X is like tensor it gives us long or float 64 we need to convert into flaot 32
y_tensor = torch.tensor(y, dtype=torch.float32)

In [ ]:
#splitting into 80 20 80-training 20-testing
split_index = int(0.8 * len(X_tensor))
X_train = X_tensor[:split_index]
X_test = X_tensor[split_index:]    # copied from daniel brouk
y_train = y_tensor[:split_index]
y_test = y_tensor[split_index:]

In [ ]:
# Define model
class PetrolPricePredictor(nn.Module):
    def __init__(self):
        super(PetrolPricePredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.ReLU()
        )

    def forward(self, x):
        return self.model(x)

model = PetrolPricePredictor()

In [ ]:
# Loss function or criterion and optimizer
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Training loop
for epoch in range(300):
    model.train()
    output = model(X_train)
    loss = criterion(output, y_train)

    optimizer.zero_grad()
    loss.backward() #it is like to improve if the result the answer is wrong then it sets automatically
    optimizer.step() # this backward will cahnge the values if we want changed data we need to use this thing

    if (epoch + 1) % 50 == 0: # i set this for check this 300 times like first 50 then 100 and 150............
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 50, Loss: 67.5034
Epoch 100, Loss: 67.5034
Epoch 150, Loss: 67.5034
Epoch 200, Loss: 67.5034
Epoch 250, Loss: 67.5034
Epoch 300, Loss: 67.5034


In [ ]:
# Evaluation
model.eval() # this will stop traning then it starts predicting
with torch.inference_mode(): #start predicting
    predictions = model(X_test)
    for i in range(5):
        actual = y_test[i].item()
        predicted = predictions[i].item()
        print(f"Predicted: {predicted:.2f} | Actual: {actual:.2f}")

Predicted: 0.00 | Actual: 73.16
Predicted: 0.00 | Actual: 73.31
Predicted: 0.00 | Actual: 73.38
Predicted: 0.00 | Actual: 73.38
Predicted: 0.00 | Actual: 73.37


In [ ]:
mape = torch.mean(torch.abs((y_test - predictions) / y_test)) * 100
print(f"\nMean Absolute Percentage Error (MAPE): {mape.item():.2f}%")


Mean Absolute Percentage Error (MAPE): 100.00%
